# Downloads & Initialization

In [ ]:
from google.colab import drive
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.models import resnet18, resnet50
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np

In [ ]:
! gdown 1aHB-jzc3g4zUOBVEokDmpC2u8o__3vak
! unzip -q /content/MURA-v1.1.zip

Downloading...
From: https://drive.google.com/uc?id=1aHB-jzc3g4zUOBVEokDmpC2u8o__3vak
To: /content/MURA-v1.1.zip
100% 3.38G/3.38G [00:31<00:00, 107MB/s]


# Data formatting & Analysis

## Train data

In [ ]:
train_df = pd.read_csv('/content/MURA-v1.1/train_labeled_studies.csv', header=None, names=['path', 'label'])

# Filter relevant bones
filtered_df_train = train_df[train_df['path'].str.contains('XR_SHOULDER|XR_HAND|XR_ELBOW')].reset_index(drop=True)

# Add bone type column
filtered_df_train['bone_type'] = filtered_df_train['path'].apply(lambda p: p.split('/')[2])

# Build df of full paths
full_paths_train = []
for _, row in filtered_df_train.iterrows():
    files = os.listdir(row['path'])
    for f in files:
        full_paths_train.append([os.path.join(row['path'], f), row['label'], row['bone_type']])

# Convert to DF
full_paths_train_df = pd.DataFrame(full_paths_train, columns=['full_path', 'label', 'bone_type'])
full_paths_train_df

,full_path,label,bone_type
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER
2,MURA-v1.1/train/XR_SHOULDER/patient00001/study...,1,XR_SHOULDER
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1,XR_SHOULDER
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...,1,XR_SHOULDER
...,...,...,...
18848,MURA-v1.1/train/XR_HAND/patient11183/study1_ne...,0,XR_HAND
18849,MURA-v1.1/train/XR_HAND/patient11183/study1_ne...,0,XR_HAND
18850,MURA-v1.1/train/XR_HAND/patient11184/study1_ne...,0,XR_HAND
18851,MURA-v1.1/train/XR_HAND/patient11184/study1_ne...,0,XR_HAND


In [ ]:
full_paths_train_df, full_paths_test_df, _, _ = train_test_split(full_paths_train_df, full_paths_train_df, test_size=0.2, random_state=777)

In [ ]:
# Generate DF with number of files per bone type
train_counts_agg = full_paths_train_df.groupby(['bone_type', 'label']).count().reset_index()
train_counts_agg = train_counts_agg.rename({'full_path': 'image_count'}, axis=1)
train_counts_agg

,bone_type,label,image_count
0,XR_ELBOW,0,2331
1,XR_ELBOW,1,1604
2,XR_HAND,0,3262
3,XR_HAND,1,1174
4,XR_SHOULDER,0,3374
5,XR_SHOULDER,1,3337


In [ ]:
fig_train = px.histogram(train_counts_agg, x="bone_type", y="image_count",
                         color='label', barmode='group',
                         height=400, text_auto=True, title='Train - Data distribution per bone type')
fig_train.show()

## Validation data

In [ ]:
val_df = pd.read_csv('/content/MURA-v1.1/valid_labeled_studies.csv', header=None, names=['path', 'label'])

# Filter relevant bones
val_df_filtered = val_df[val_df['path'].str.contains('XR_SHOULDER|XR_HAND|XR_ELBOW')].reset_index(drop=True)

# Add bone type column
val_df_filtered['bone_type'] = val_df_filtered['path'].apply(lambda p: p.split('/')[2])

# Build df of full paths
full_paths_val = []
for _, row in val_df_filtered.iterrows():
    files = os.listdir(row['path'])
    for f in files:
        full_paths_val.append([os.path.join(row['path'], f), row['label'], row['bone_type']])

# Convert to DF
full_paths_val_df = pd.DataFrame(full_paths_val, columns=['full_path', 'label', 'bone_type'])
full_paths_val_df

,full_path,label,bone_type
0,MURA-v1.1/valid/XR_HAND/patient11497/study1_po...,1,XR_HAND
1,MURA-v1.1/valid/XR_HAND/patient11497/study1_po...,1,XR_HAND
2,MURA-v1.1/valid/XR_HAND/patient11497/study1_po...,1,XR_HAND
3,MURA-v1.1/valid/XR_HAND/patient11498/study1_po...,1,XR_HAND
4,MURA-v1.1/valid/XR_HAND/patient11498/study1_po...,1,XR_HAND
...,...,...,...
1483,MURA-v1.1/valid/XR_ELBOW/patient11205/study1_n...,0,XR_ELBOW
1484,MURA-v1.1/valid/XR_ELBOW/patient11881/study1_n...,0,XR_ELBOW
1485,MURA-v1.1/valid/XR_ELBOW/patient11881/study1_n...,0,XR_ELBOW
1486,MURA-v1.1/valid/XR_ELBOW/patient11334/study1_n...,0,XR_ELBOW


In [ ]:
# Generate DF with number of files per bone type
val_counts_agg = full_paths_val_df.groupby(['bone_type','label']).count().reset_index()
val_counts_agg = val_counts_agg.rename({'full_path': 'image_count'}, axis=1)
val_counts_agg

,bone_type,label,image_count
0,XR_ELBOW,0,235
1,XR_ELBOW,1,230
2,XR_HAND,0,271
3,XR_HAND,1,189
4,XR_SHOULDER,0,285
5,XR_SHOULDER,1,278


In [ ]:
fig_val = px.histogram(val_counts_agg, x="bone_type", y="image_count",
                       color='label', barmode='group',
                       height=400, text_auto=True, title='Validation - Data distribution per bone type')
fig_val.show()

## Test data

In [ ]:
# Generate DF with number of files per bone type
test_counts_agg = full_paths_test_df.groupby(['bone_type', 'label']).count().reset_index()
test_counts_agg = test_counts_agg.rename({'full_path': 'image_count'}, axis=1)
test_counts_agg

,bone_type,label,image_count
0,XR_ELBOW,0,594
1,XR_ELBOW,1,402
2,XR_HAND,0,797
3,XR_HAND,1,310
4,XR_SHOULDER,0,837
5,XR_SHOULDER,1,831


In [ ]:
fig_test = px.histogram(test_counts_agg, x="bone_type", y="image_count",
                        color='label', barmode='group',
                        height=400, text_auto=True, title='Test - Data distribution per bone type')
fig_test.show()

# Data Helpers

In [ ]:
BASE_PATH = '/content/models'

In [ ]:
# Define MURAv1.1 dataset
class MURA(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        if self.transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.Grayscale(num_output_channels=1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485], std=[0.229])
            ])

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        image = Image.open(row['full_path']).convert('RGB')
        image = self.transform(image)

        return image, row['label']

In [ ]:
class TrainerMURA():
    def __init__(self, model, train_data_df, val_data_df, test_data_df,
                 transform=None, batch_size=32, train_frac=1, seed=777, lr=0.001):
        # Take only frac of the trainset
        train_data_df_frac = train_data_df.sample(frac=train_frac, random_state=seed)

        # Create a custom dataset instance
        self.train_data = MURA(train_data_df_frac, transform)
        self.val_data = MURA(val_data_df, transform)
        self.test_data = MURA(test_data_df, transform)

        # Create training and validation data loaders
        self.batch_size = batch_size
        self.train_loader = DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=0)
        self.val_loader = DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, num_workers=0)
        self.test_loader = DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, num_workers=0)

        # Set the device to GPU if available, otherwise use CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Create an instance of the DenseNet-169 model
        self.model = model
        self.model.to(self.device)

        # Define the loss function and optimizer
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

        self.val_loss = []
        self.val_acc = []

    def load_model(self, ckpt_path):
        data = torch.load(ckpt_path)
        self.model.load_state_dict(data['model'])
        self.optimizer.load_state_dict(data['optimizer'])
        self.val_loss = data['loss']
        self.val_acc = data['acc']
        self.model.eval()

    def save_model(self, ckpt_path):
        data = {
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'loss': self.val_loss,
            'acc': self.val_acc
        }
        torch.save(data, ckpt_path)

    def __epoch(self, data_loader, train=True):
        if train:
            self.model.train()
        else:
            self.model.eval()

        loss_value = 0.0
        correct = 0

        with torch.set_grad_enabled(train):
            for images, labels in data_loader:
                images = images.to(self.device)
                labels = labels.to(self.device)

                if train:
                    self.optimizer.zero_grad()

                outputs = self.model(images)
                loss = self.criterion(outputs.squeeze(), labels.float())

                if train:
                    loss.backward()
                    self.optimizer.step()

                loss_value += loss.item() * images.size(0)
                if outputs.round().squeeze().shape != labels.shape:
                     print(outputs.shape, outputs.round().squeeze().shape, labels.shape)
                correct += (outputs.round().squeeze() == labels).sum().item()

        loss_value = loss_value / len(data_loader.dataset)
        accuracy = correct / len(data_loader.dataset)
        return loss_value, accuracy

    def train(self, ckpt_dir, ckpt_name='model', epochs=10, start_epoch=0):
        try:
            os.makedirs(ckpt_dir, exist_ok=True)
            for epoch in range(start_epoch, start_epoch + epochs):
                # Training
                train_loss, train_acc = self.__epoch(self.train_loader, train=True)

                # Validation
                val_loss, val_acc = self.__epoch(self.val_loader, train=False)
                self.val_loss.append(val_loss)
                self.val_acc.append(val_acc)

                print(f'Epoch {epoch+1}/{epochs} - train_loss={train_loss:.4f} train_acc={train_acc:.4f} val_loss={val_loss:.4f} val_acc={val_acc:.4f}')

                # Save the trained model
                self.save_model(os.path.join(ckpt_dir, f'{ckpt_name}_{epoch}.pth'))
        except KeyboardInterrupt:
            pass

        # Save the trained model
        self.save_model(os.path.join(ckpt_dir, f'{ckpt_name}.pth'))
        print('Done.')

    def test(self):
        _, acc = self.__epoch(self.test_loader, train=False)
        return acc

In [ ]:
def plot_train(data, name):
    epochs = len(data.val_loss)
    x = list(range(1, epochs + 1))
    titles = ['Loss at each epoch', 'Accuracy at each epoch']

    fig = make_subplots(rows=1, cols=2, subplot_titles=titles)

    fig_loss = px.line(x=x, y=data.val_loss, labels={'x': 'Epoch', 'y': 'Loss'}, markers=True)
    fig_acc = px.line(x=x, y=data.val_acc, labels={'x': 'Epoch', 'y': 'Accuracy'}, markers=True)

    fig.add_trace(fig_loss['data'][0], row=1, col=1)
    fig.add_trace(fig_acc['data'][0], row=1, col=2)

    fig['layout']['xaxis']['title'] = 'Epochs'
    fig['layout']['xaxis2']['title'] = 'Epochs'
    fig['layout']['yaxis']['title'] = 'Loss'
    fig['layout']['yaxis2']['title'] = 'Accuracy'
    fig.update_layout(title_text=f'{name}', title_x=0.5)
    fig.show()

# Naive Baseline

## DenseNet169

In [ ]:
# Define the DenseNet-169 model
class DenseNet169(nn.Module):
    def __init__(self, num_classes=1, block_layers=[6, 12, 24, 16], block_growth=32):
        super(DenseNet169, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.block_layers = block_layers
        self.block_growth = block_growth

        num_features = 64
        self.dense_blocks = [self._make_dense_block(self.block_layers[0], num_features)]
        num_features = num_features + self.block_growth * self.block_layers[0]

        for block_layer in self.block_layers[1:]:
            self.dense_blocks.append(Transition(num_features))
            num_features = num_features // 2
            self.dense_blocks.append(self._make_dense_block(block_layer, num_features))
            num_features = num_features + self.block_growth * block_layer
        self.dense_blocks = nn.ModuleList(self.dense_blocks)

        self.batch_norm = nn.BatchNorm2d(num_features)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(num_features, num_classes)

        # Weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)

        for block in self.dense_blocks:
            x = block(x)

        x = self.batch_norm(x)
        x = F.relu(x, inplace=True)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = F.sigmoid(x)
        return x

    def _make_dense_block(self, num_layers, num_input_features):
        layers = []
        for _ in range(num_layers):
            layers.append(Bottleneck(num_input_features))
            num_input_features += self.block_growth
        return nn.Sequential(*layers)

# Define the Bottleneck block used in DenseNet
class Bottleneck(nn.Module):
    def __init__(self, num_input_features):
        super(Bottleneck, self).__init__()

        self.features = nn.Sequential(
            nn.BatchNorm2d(num_input_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_input_features, 128, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 32, kernel_size=3, stride=1, padding=1, bias=False)
        )

    def forward(self, x):
        out = self.features(x)
        out = torch.cat((x, out), 1)
        return out

# Define the Transition block used in DenseNet
class Transition(nn.Module):
    def __init__(self, num_input_features):
        super(Transition, self).__init__()

        self.features = nn.Sequential(
            nn.BatchNorm2d(num_input_features),
            nn.ReLU(inplace=True),
            nn.Conv2d(num_input_features, num_input_features//2, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.features(x)

## XR_HAND

In [ ]:
hand_name = 'XR_HAND'
hand_train_data_df = full_paths_train_df[full_paths_train_df['bone_type'] == hand_name]
hand_val_data_df = full_paths_val_df[full_paths_val_df['bone_type'] == hand_name]
hand_test_data_df = full_paths_test_df[full_paths_test_df['bone_type'] == hand_name]

In [ ]:
hand_densenet = TrainerMURA(DenseNet169(), hand_train_data_df, hand_val_data_df, hand_test_data_df, lr=0.0001)

In [ ]:
run_id = 5
hand_ckpt_path = os.path.join(BASE_PATH, 'DenseNet', hand_name, str(run_id))
hand_densenet.train(hand_ckpt_path, epochs=10)

Epoch 1/10 - train_loss=0.7152 train_acc=0.7360 val_loss=0.6928 val_acc=0.5891
Epoch 2/10 - train_loss=0.6947 train_acc=0.7376 val_loss=0.6873 val_acc=0.6109
Epoch 3/10 - train_loss=0.6913 train_acc=0.7493 val_loss=0.6812 val_acc=0.6283
Epoch 4/10 - train_loss=0.6874 train_acc=0.7583 val_loss=0.6833 val_acc=0.6130
Epoch 5/10 - train_loss=0.6867 train_acc=0.7595 val_loss=0.6788 val_acc=0.6283
Epoch 6/10 - train_loss=0.6852 train_acc=0.7613 val_loss=0.6795 val_acc=0.6261
Epoch 7/10 - train_loss=0.6847 train_acc=0.7617 val_loss=0.6776 val_acc=0.6304
Epoch 8/10 - train_loss=0.6825 train_acc=0.7669 val_loss=0.6794 val_acc=0.6261
Epoch 9/10 - train_loss=0.6830 train_acc=0.7653 val_loss=0.6802 val_acc=0.6217
Epoch 10/10 - train_loss=0.6818 train_acc=0.7676 val_loss=0.6777 val_acc=0.6304
Done.


In [ ]:
hand_test_acc = hand_densenet.test()
print(f'Test accuracy on XR_SHOULDER: {100*hand_test_acc:.3f}%')

Test accuracy on XR_SHOULDER: 75.610%


In [ ]:
plot_train(hand_densenet, f'{hand_name} - DenseNet')

## XR_SHOULDER

In [ ]:
shoulder_name = 'XR_SHOULDER'
shoulder_train_data_df = full_paths_train_df[full_paths_train_df['bone_type'] == shoulder_name]
shoulder_val_data_df = full_paths_val_df[full_paths_val_df['bone_type'] == shoulder_name]
shoulder_test_data_df = full_paths_test_df[full_paths_test_df['bone_type'] == shoulder_name]

In [ ]:
shoulder_densenet = TrainerMURA(DenseNet169(), shoulder_train_data_df, shoulder_val_data_df, shoulder_test_data_df, lr=0.00001)

In [ ]:
run_id = 5
shoulder_ckpt_path = os.path.join(BASE_PATH, 'DenseNet', shoulder_name, str(run_id))
shoulder_densenet.train(shoulder_ckpt_path, epochs=10)

Epoch 1/10 - train_loss=0.7007 train_acc=0.5278 val_loss=0.6875 val_acc=0.5542
Epoch 2/10 - train_loss=0.6817 train_acc=0.5668 val_loss=0.6706 val_acc=0.6075
Epoch 3/10 - train_loss=0.6669 train_acc=0.6162 val_loss=0.6544 val_acc=0.6412
Epoch 4/10 - train_loss=0.6540 train_acc=0.6474 val_loss=0.6482 val_acc=0.6767
Epoch 5/10 - train_loss=0.6442 train_acc=0.6811 val_loss=0.6490 val_acc=0.6643
Epoch 6/10 - train_loss=0.6389 train_acc=0.6880 val_loss=0.6420 val_acc=0.6803
Epoch 7/10 - train_loss=0.6333 train_acc=0.7015 val_loss=0.6365 val_acc=0.7052
Epoch 8/10 - train_loss=0.6285 train_acc=0.7166 val_loss=0.6397 val_acc=0.6892
Epoch 9/10 - train_loss=0.6254 train_acc=0.7222 val_loss=0.6439 val_acc=0.6643
Epoch 10/10 - train_loss=0.6225 train_acc=0.7273 val_loss=0.6444 val_acc=0.6874
Done.


In [ ]:
shoulder_test_acc = shoulder_densenet.test()
print(f'Test accuracy on XR_SHOULDER: {100*shoulder_test_acc:.3f}%')

Test accuracy on XR_SHOULDER: 66.547%


In [ ]:
plot_train(shoulder_densenet, f'{shoulder_name} - DenseNet')

## XR_ELBOW

In [ ]:
elbow_name = 'XR_ELBOW'
elbow_train_data_df = full_paths_train_df[full_paths_train_df['bone_type'] == elbow_name]
elbow_val_data_df = full_paths_val_df[full_paths_val_df['bone_type'] == elbow_name]
elbow_test_data_df = full_paths_test_df[full_paths_test_df['bone_type'] == elbow_name]

In [ ]:
elbow_densenet = TrainerMURA(DenseNet169(), elbow_train_data_df, elbow_val_data_df, elbow_test_data_df, lr=0.00001)

In [ ]:
run_id = 5
elbow_ckpt_path = os.path.join(BASE_PATH, 'DenseNet', elbow_name, str(run_id))
elbow_densenet.train(elbow_ckpt_path, epochs=10)

Epoch 1/10 - train_loss=0.7388 train_acc=0.6178 val_loss=0.6947 val_acc=0.5290
Epoch 2/10 - train_loss=0.7131 train_acc=0.6320 val_loss=0.6864 val_acc=0.5441
Epoch 3/10 - train_loss=0.6999 train_acc=0.6391 val_loss=0.6789 val_acc=0.5527
Epoch 4/10 - train_loss=0.6895 train_acc=0.6663 val_loss=0.6708 val_acc=0.5871
Epoch 5/10 - train_loss=0.6818 train_acc=0.6808 val_loss=0.6664 val_acc=0.5892
Epoch 6/10 - train_loss=0.6774 train_acc=0.6867 val_loss=0.6639 val_acc=0.5871
Epoch 7/10 - train_loss=0.6711 train_acc=0.6882 val_loss=0.6579 val_acc=0.6022
Epoch 8/10 - train_loss=0.6646 train_acc=0.7024 val_loss=0.6444 val_acc=0.6409
Epoch 9/10 - train_loss=0.6616 train_acc=0.7044 val_loss=0.6618 val_acc=0.5828
Epoch 10/10 - train_loss=0.6568 train_acc=0.7161 val_loss=0.6429 val_acc=0.6538
Done.


In [ ]:
elbow_test_acc = elbow_densenet.test()
print(f'Test accuracy on XR_SHOULDER: {100*elbow_test_acc:.3f}%')

Test accuracy on XR_SHOULDER: 71.486%


In [ ]:
plot_train(elbow_densenet, f'{elbow_name} - DenseNet')

In [ ]:
names = [hand_name, shoulder_name, elbow_name]
res_baselines = np.array([hand_test_acc, shoulder_test_acc, elbow_test_acc])
res_baselines = np.round(100 * res_baselines, 3)
fig_baselines = px.histogram(x=names, y=res_baselines, color=names, height=400, text_auto=True,
                             labels={"y": "Accuracy (%)", "x": "Bone Type"},
                             title='Baseline accuracy on test set for each bone type')
fig_baselines.show()

# Representation Learning

## SimCLR

In [ ]:
# Define the SimCLR model
class SimCLR(nn.Module):
    def __init__(self, projection_dim=1):
        super(SimCLR, self).__init__()
        # Set the dimension of the projected representation
        self.projection_dim = projection_dim

        self.base_encoder = resnet18()
        self.base_encoder = nn.Sequential(*list(self.base_encoder.children())[:-1])
        self.projection_fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, projection_dim)
        )

    def forward(self, x):
        x = self.base_encoder(x)
        x = x.view(x.size(0), -1)
        x = self.projection_fc(x)
        x = F.sigmoid(x)
        return x

In [ ]:
transform_simclr = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## XR_HAND

### 1% of the data

In [ ]:
hand_simclr_1 = TrainerMURA(SimCLR(), hand_train_data_df, hand_val_data_df, hand_test_data_df,
                            transform_simclr, train_frac=0.01, lr=0.000001)

In [ ]:
run_id = 2
hand_ckpt_path_1 = os.path.join(BASE_PATH, 'SimCLR_1', hand_name, str(run_id))
hand_simclr_1.train(hand_ckpt_path_1, epochs=10)

Epoch 1/10 - train_loss=0.8545 train_acc=0.2727 val_loss=0.7718 val_acc=0.4087
Epoch 2/10 - train_loss=0.8542 train_acc=0.2500 val_loss=0.7691 val_acc=0.4283
Epoch 3/10 - train_loss=0.8535 train_acc=0.2955 val_loss=0.7685 val_acc=0.5065
Epoch 4/10 - train_loss=0.8529 train_acc=0.2727 val_loss=0.7680 val_acc=0.5304
Epoch 5/10 - train_loss=0.8522 train_acc=0.2955 val_loss=0.7677 val_acc=0.5870
Epoch 6/10 - train_loss=0.8515 train_acc=0.3182 val_loss=0.7674 val_acc=0.5978
Epoch 7/10 - train_loss=0.8510 train_acc=0.3636 val_loss=0.7673 val_acc=0.5957
Epoch 8/10 - train_loss=0.8503 train_acc=0.3409 val_loss=0.7673 val_acc=0.6022
Epoch 9/10 - train_loss=0.8498 train_acc=0.4091 val_loss=0.7672 val_acc=0.5870
Epoch 10/10 - train_loss=0.8493 train_acc=0.4773 val_loss=0.7671 val_acc=0.5804
Done.


In [ ]:
hand_test_acc_1 = hand_simclr_1.test()
print(f'Test accuracy on XR_HAND: {100*hand_test_acc_1:.3f}%')

Test accuracy on XR_HAND: 68.654%


In [ ]:
plot_train(hand_simclr_1, f'{hand_name} - SimCLR 1%')

### 10% of the data

In [ ]:
hand_simclr_10 = TrainerMURA(SimCLR(), hand_train_data_df, hand_val_data_df, hand_test_data_df,
                             transform_simclr, train_frac=0.1, lr=0.000001)

In [ ]:
run_id = 2
hand_ckpt_path_10 = os.path.join(BASE_PATH, 'SimCLR_10', hand_name, str(run_id))
hand_simclr_10.train(hand_ckpt_path_10, epochs=10)

Epoch 1/10 - train_loss=0.8211 train_acc=0.7342 val_loss=0.7630 val_acc=0.5891
Epoch 2/10 - train_loss=0.8178 train_acc=0.7342 val_loss=0.7601 val_acc=0.5891
Epoch 3/10 - train_loss=0.8145 train_acc=0.7342 val_loss=0.7566 val_acc=0.5891
Epoch 4/10 - train_loss=0.8118 train_acc=0.7342 val_loss=0.7534 val_acc=0.5891
Epoch 5/10 - train_loss=0.8088 train_acc=0.7342 val_loss=0.7511 val_acc=0.5891
Epoch 6/10 - train_loss=0.8059 train_acc=0.7342 val_loss=0.7491 val_acc=0.5891
Epoch 7/10 - train_loss=0.8037 train_acc=0.7342 val_loss=0.7476 val_acc=0.5891
Epoch 8/10 - train_loss=0.8009 train_acc=0.7342 val_loss=0.7461 val_acc=0.5891
Epoch 9/10 - train_loss=0.7983 train_acc=0.7342 val_loss=0.7448 val_acc=0.5891
Epoch 10/10 - train_loss=0.7956 train_acc=0.7342 val_loss=0.7432 val_acc=0.5891
Done.


In [ ]:
hand_test_acc_10 = hand_simclr_10.test()
print(f'Test accuracy on XR_HAND: {100*hand_test_acc_10:.3f}%')

Test accuracy on XR_HAND: 71.996%


In [ ]:
plot_train(hand_simclr_10, f'{hand_name} - SimCLR 10%')

### 100% of the data

In [ ]:
hand_simclr_100 = TrainerMURA(SimCLR(), hand_train_data_df, hand_val_data_df, hand_test_data_df,
                              transform_simclr, lr=0.00000005)

In [ ]:
run_id = 2
hand_ckpt_path_100 = os.path.join(BASE_PATH, 'SimCLR_100', hand_name, str(run_id))
hand_simclr_100.train(hand_ckpt_path_100, epochs=10)

Epoch 1/10 - train_loss=0.8405 train_acc=0.5399 val_loss=0.7679 val_acc=0.4848
Epoch 2/10 - train_loss=0.8389 train_acc=0.5976 val_loss=0.7671 val_acc=0.5087
Epoch 3/10 - train_loss=0.8372 train_acc=0.6382 val_loss=0.7662 val_acc=0.5435
Epoch 4/10 - train_loss=0.8356 train_acc=0.6718 val_loss=0.7652 val_acc=0.5587
Epoch 5/10 - train_loss=0.8340 train_acc=0.6966 val_loss=0.7641 val_acc=0.5717
Epoch 6/10 - train_loss=0.8324 train_acc=0.7115 val_loss=0.7633 val_acc=0.5826
Epoch 7/10 - train_loss=0.8308 train_acc=0.7254 val_loss=0.7625 val_acc=0.5826
Epoch 8/10 - train_loss=0.8292 train_acc=0.7297 val_loss=0.7619 val_acc=0.5848
Epoch 9/10 - train_loss=0.8277 train_acc=0.7335 val_loss=0.7612 val_acc=0.5870
Epoch 10/10 - train_loss=0.8261 train_acc=0.7340 val_loss=0.7602 val_acc=0.5891
Done.


In [ ]:
hand_test_acc_100 = hand_simclr_100.test()
print(f'Test accuracy on XR_HAND: {100*hand_test_acc_100:.3f}%')

Test accuracy on XR_HAND: 72.087%


In [ ]:
plot_train(hand_simclr_100, f'{hand_name} - SimCLR 100%')

## XR_SHOULDER

### 1% of the data

In [ ]:
shoulder_simclr_1 = TrainerMURA(SimCLR(), shoulder_train_data_df, shoulder_val_data_df, shoulder_test_data_df,
                                  transform_simclr, train_frac=0.01, lr=0.00001)

In [ ]:
run_id = 2
shoulder_ckpt_path_1 = os.path.join(BASE_PATH, 'SimCLR_1', shoulder_name, str(run_id))
shoulder_simclr_1.train(shoulder_ckpt_path_1, epochs=10)

Epoch 1/10 - train_loss=0.7282 train_acc=0.5672 val_loss=0.7248 val_acc=0.5062
Epoch 2/10 - train_loss=0.7220 train_acc=0.6418 val_loss=0.7255 val_acc=0.5062
Epoch 3/10 - train_loss=0.7175 train_acc=0.6716 val_loss=0.7259 val_acc=0.5062
Epoch 4/10 - train_loss=0.7135 train_acc=0.6418 val_loss=0.7261 val_acc=0.5080
Epoch 5/10 - train_loss=0.7119 train_acc=0.6567 val_loss=0.7266 val_acc=0.4671
Epoch 6/10 - train_loss=0.7094 train_acc=0.6567 val_loss=0.7268 val_acc=0.4618
Epoch 7/10 - train_loss=0.7049 train_acc=0.6567 val_loss=0.7270 val_acc=0.4476
Epoch 8/10 - train_loss=0.7040 train_acc=0.6567 val_loss=0.7269 val_acc=0.4600
Epoch 9/10 - train_loss=0.7009 train_acc=0.6418 val_loss=0.7269 val_acc=0.4600
Epoch 10/10 - train_loss=0.6979 train_acc=0.7164 val_loss=0.7265 val_acc=0.4565
Done.


In [ ]:
shoulder_test_acc_1 = shoulder_simclr_1.test()
print(f'Test accuracy on XR_SHOULDER: {100*shoulder_test_acc_1:.3f}%')

Test accuracy on XR_SHOULDER: 50.240%


In [ ]:
plot_train(shoulder_simclr_1, f'{shoulder_name} - SimCLR 1%')

### 10% of the data

In [ ]:
shoulder_simclr_10 = TrainerMURA(SimCLR(), shoulder_train_data_df, shoulder_val_data_df, shoulder_test_data_df,
                                 transform_simclr, train_frac=0.1, lr=0.00001)

In [ ]:
run_id = 2
shoulder_ckpt_path_10 = os.path.join(BASE_PATH, 'SimCLR_10', shoulder_name, str(run_id))
shoulder_simclr_10.train(shoulder_ckpt_path_10, epochs=10)

Epoch 1/10 - train_loss=0.7210 train_acc=0.5112 val_loss=0.7181 val_acc=0.5062
Epoch 2/10 - train_loss=0.7101 train_acc=0.5097 val_loss=0.7109 val_acc=0.5062
Epoch 3/10 - train_loss=0.7020 train_acc=0.5097 val_loss=0.7027 val_acc=0.5062
Epoch 4/10 - train_loss=0.6973 train_acc=0.5097 val_loss=0.6981 val_acc=0.5062
Epoch 5/10 - train_loss=0.6917 train_acc=0.5097 val_loss=0.6947 val_acc=0.5062
Epoch 6/10 - train_loss=0.6882 train_acc=0.5112 val_loss=0.6916 val_acc=0.5115
Epoch 7/10 - train_loss=0.6808 train_acc=0.5291 val_loss=0.6881 val_acc=0.5204
Epoch 8/10 - train_loss=0.6744 train_acc=0.5723 val_loss=0.6840 val_acc=0.5560
Epoch 9/10 - train_loss=0.6670 train_acc=0.6021 val_loss=0.6809 val_acc=0.5737
Epoch 10/10 - train_loss=0.6534 train_acc=0.6542 val_loss=0.6792 val_acc=0.5755
Done.


In [ ]:
shoulder_test_acc_10 = shoulder_simclr_10.test()
print(f'Test accuracy on XR_SHOULDER: {100*shoulder_test_acc_10:.3f}%')

Test accuracy on XR_SHOULDER: 56.775%


In [ ]:
plot_train(shoulder_simclr_10, f'{shoulder_name} - SimCLR 10%')

### 100% of the data

In [ ]:
shoulder_simclr_100 = TrainerMURA(SimCLR(), shoulder_train_data_df, shoulder_val_data_df, shoulder_test_data_df,
                                  transform_simclr, lr=0.00001)

In [ ]:
run_id = 2
shoulder_ckpt_path_100 = os.path.join(BASE_PATH, 'SimCLR_100', shoulder_name, str(run_id))
shoulder_simclr_100.train(shoulder_ckpt_path_100, epochs=10)

Epoch 1/10 - train_loss=0.6977 train_acc=0.5065 val_loss=0.6817 val_acc=0.5382
Epoch 2/10 - train_loss=0.6723 train_acc=0.5777 val_loss=0.6591 val_acc=0.6181
Epoch 3/10 - train_loss=0.6524 train_acc=0.6500 val_loss=0.6511 val_acc=0.6323
Epoch 4/10 - train_loss=0.6415 train_acc=0.6765 val_loss=0.6448 val_acc=0.6785
Epoch 5/10 - train_loss=0.6350 train_acc=0.6950 val_loss=0.6381 val_acc=0.6838
Epoch 6/10 - train_loss=0.6255 train_acc=0.7157 val_loss=0.6397 val_acc=0.6909
Epoch 7/10 - train_loss=0.6159 train_acc=0.7428 val_loss=0.6456 val_acc=0.6803
Epoch 8/10 - train_loss=0.6058 train_acc=0.7686 val_loss=0.6464 val_acc=0.6696
Epoch 9/10 - train_loss=0.5961 train_acc=0.7872 val_loss=0.6489 val_acc=0.6821
Epoch 10/10 - train_loss=0.5868 train_acc=0.8118 val_loss=0.6369 val_acc=0.6874
Done.


In [ ]:
shoulder_test_acc_100 = shoulder_simclr_100.test()
print(f'Test accuracy on XR_SHOULDER: {100*shoulder_test_acc_100:.3f}%')

Test accuracy on XR_SHOULDER: 64.628%


In [ ]:
plot_train(shoulder_simclr_100, f'{shoulder_name} - SimCLR 100%')

## XR_ELBOW

### 1% of the data

In [ ]:
elbow_simclr_1 = TrainerMURA(SimCLR(), elbow_train_data_df, elbow_val_data_df, elbow_test_data_df,
                             transform_simclr, train_frac=0.01, lr=0.0001)

In [ ]:
run_id = 2
elbow_ckpt_path_1 = os.path.join(BASE_PATH, 'SimCLR_1', elbow_name, str(run_id))
elbow_simclr_1.train(elbow_ckpt_path_1, epochs=10)

Epoch 1/10 - train_loss=0.7761 train_acc=0.6410 val_loss=0.7369 val_acc=0.4968
Epoch 2/10 - train_loss=0.7396 train_acc=0.6667 val_loss=0.7253 val_acc=0.4774
Epoch 3/10 - train_loss=0.7137 train_acc=0.6667 val_loss=0.7183 val_acc=0.5118
Epoch 4/10 - train_loss=0.6955 train_acc=0.6923 val_loss=0.7129 val_acc=0.4946
Epoch 5/10 - train_loss=0.6794 train_acc=0.7436 val_loss=0.7081 val_acc=0.4946
Epoch 6/10 - train_loss=0.6807 train_acc=0.7436 val_loss=0.7041 val_acc=0.4925
Epoch 7/10 - train_loss=0.6607 train_acc=0.7692 val_loss=0.7024 val_acc=0.5011
Epoch 8/10 - train_loss=0.6511 train_acc=0.8462 val_loss=0.7016 val_acc=0.5011
Epoch 9/10 - train_loss=0.6422 train_acc=0.8462 val_loss=0.7029 val_acc=0.5054
Epoch 10/10 - train_loss=0.6320 train_acc=0.8462 val_loss=0.7051 val_acc=0.5183
Done.


In [ ]:
elbow_test_acc_1 = elbow_simclr_1.test()
print(f'Test accuracy on XR_SHOULDER: {100*elbow_test_acc_1:.3f}%')

Test accuracy on XR_SHOULDER: 60.341%


In [ ]:
plot_train(elbow_simclr_1, f'{elbow_name} - SimCLR 1%')

### 10% of the data

In [ ]:
elbow_simclr_10 = TrainerMURA(SimCLR(), elbow_train_data_df, elbow_val_data_df, elbow_test_data_df,
                              transform_simclr, train_frac=0.1, lr=0.00005)

In [ ]:
run_id = 2
elbow_ckpt_path_10 = os.path.join(BASE_PATH, 'SimCLR_10', elbow_name, str(run_id))
elbow_simclr_10.train(elbow_ckpt_path_10, epochs=10)

Epoch 1/10 - train_loss=0.7176 train_acc=0.5228 val_loss=0.7076 val_acc=0.4968
Epoch 2/10 - train_loss=0.6799 train_acc=0.5838 val_loss=0.6918 val_acc=0.5118
Epoch 3/10 - train_loss=0.6693 train_acc=0.6320 val_loss=0.6884 val_acc=0.5333
Epoch 4/10 - train_loss=0.6654 train_acc=0.6929 val_loss=0.7046 val_acc=0.5548
Epoch 5/10 - train_loss=0.6590 train_acc=0.6827 val_loss=0.6957 val_acc=0.5892
Epoch 6/10 - train_loss=0.6529 train_acc=0.6853 val_loss=0.6910 val_acc=0.5312
Epoch 7/10 - train_loss=0.6250 train_acc=0.7640 val_loss=0.6822 val_acc=0.5957
Epoch 8/10 - train_loss=0.6099 train_acc=0.8274 val_loss=0.7057 val_acc=0.5892
Epoch 9/10 - train_loss=0.5813 train_acc=0.8858 val_loss=0.6688 val_acc=0.6065
Epoch 10/10 - train_loss=0.5670 train_acc=0.9086 val_loss=0.7076 val_acc=0.6043
Done.


In [ ]:
elbow_test_acc_10 = elbow_simclr_10.test()
print(f'Test accuracy on XR_SHOULDER: {100*elbow_test_acc_10:.3f}%')

Test accuracy on XR_SHOULDER: 60.643%


In [ ]:
plot_train(elbow_simclr_10, f'{elbow_name} - SimCLR 10%')

### 100% of the data

In [ ]:
elbow_simclr_100 = TrainerMURA(SimCLR(), elbow_train_data_df, elbow_val_data_df, elbow_test_data_df,
                               transform_simclr, lr=0.00001)

In [ ]:
run_id = 2
elbow_ckpt_path_100 = os.path.join(BASE_PATH, 'SimCLR_100', elbow_name, str(run_id))
elbow_simclr_100.train(elbow_ckpt_path_100, epochs=10)

Epoch 1/10 - train_loss=0.7232 train_acc=0.5919 val_loss=0.6897 val_acc=0.5054
Epoch 2/10 - train_loss=0.6947 train_acc=0.5936 val_loss=0.6866 val_acc=0.5075
Epoch 3/10 - train_loss=0.6851 train_acc=0.6203 val_loss=0.6716 val_acc=0.5677
Epoch 4/10 - train_loss=0.6730 train_acc=0.6640 val_loss=0.6571 val_acc=0.6129
Epoch 5/10 - train_loss=0.6645 train_acc=0.6892 val_loss=0.6597 val_acc=0.6065
Epoch 6/10 - train_loss=0.6586 train_acc=0.6994 val_loss=0.6562 val_acc=0.6086
Epoch 7/10 - train_loss=0.6572 train_acc=0.7027 val_loss=0.6519 val_acc=0.6194
Epoch 8/10 - train_loss=0.6525 train_acc=0.7161 val_loss=0.6524 val_acc=0.6258
Epoch 9/10 - train_loss=0.6459 train_acc=0.7321 val_loss=0.6598 val_acc=0.6000
Epoch 10/10 - train_loss=0.6418 train_acc=0.7454 val_loss=0.6506 val_acc=0.6280
Done.


In [ ]:
elbow_test_acc_100 = elbow_simclr_100.test()
print(f'Test accuracy on XR_SHOULDER: {100*elbow_test_acc_100:.3f}%')

Test accuracy on XR_SHOULDER: 69.980%


In [ ]:
plot_train(elbow_simclr_100, f'{elbow_name} - SimCLR 100%')

In [ ]:
percentages = ['1%', '10%', '100%']
simclr_test_res = {
    hand_name: [hand_test_acc_1, hand_test_acc_10, hand_test_acc_100],
    shoulder_name: [shoulder_test_acc_1, shoulder_test_acc_10, shoulder_test_acc_100],
    elbow_name: [elbow_test_acc_1, elbow_test_acc_10, elbow_test_acc_100]
}

simclr_res = []
for name, vals in simclr_test_res.items():
    for p, v in zip(percentages, vals):
        simclr_res.append((name, p, np.round(100 * v, 3)))

simclr_res_df = pd.DataFrame(simclr_res, columns=['Bone Type', 'Train size', 'Accuracy (%)'])
fig_simclr = px.histogram(simclr_res_df, x='Bone Type', y='Accuracy (%)', color='Train size',
                          height=400, text_auto=True, barmode='group',
                          title='SimCLR accuracy on test set for each bone type and train size')
fig_simclr.show()

In [ ]:
models = ['SimCLR (100%)', 'Baseline (100%)']
comp_test_res = {
    hand_name: [hand_test_acc_100, hand_test_acc],
    shoulder_name: [shoulder_test_acc_100, shoulder_test_acc],
    elbow_name: [elbow_test_acc_100, elbow_test_acc]
}

comp_res = []
for name, vals in comp_test_res.items():
    for m, v in zip(models, vals):
        comp_res.append((name, m, np.round(100 * v, 3)))

comp_res_df = pd.DataFrame(comp_res, columns=['Bone Type', 'Model', 'Accuracy (%)'])
fig_comp = px.histogram(comp_res_df, x='Bone Type', y='Accuracy (%)', color='Model',
                        height=400, text_auto=True, barmode='group',
                        title='SimCLR vs Baseline accuracy on test set for each bone type and 100% train size')
fig_comp.show()